# Le Titanic

<img src="https://www.scienceabc.com/wp-content/uploads/2016/04/titanic-jack-and-rose-plank-scene.webp">

## Lecture des données

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# Lecture des données d'apprentissage et de test
t = pd.read_csv("../input/titanic/train.csv")

In [ ]:
t.head().T

## Interprétation des paramètres

# Rose & Jack

*value_counts* permet de compter le nombre d'éléments par catégorie d'une série

In [ ]:
t.Sex.value_counts()      # nombre d'hommes et de femmes

In [ ]:
t.Sex.count()              # nombre total hommes+femmes

In [ ]:
t.Cabin.count()

In [ ]:
t.count()                  # Comptage par colonnes

In [ ]:
t[np.isnan(t.Age)].Survived.value_counts()

On remarque qu'il manque des valeurs pour 'age' et 'embarked' (présence de valeurs indéfinies 'NaN')

On peut définir un booléen pour abréger une caractéristique :

In [ ]:
hommes = (t.Sex=="male")

In [ ]:
t[hommes].head()        # t[hommes] est le tableau où on ne retient que lignes pour lesquelles hommes est True

On peut compter les hommes survivants ou non :

In [ ]:
t[hommes].Survived.value_counts()

## Exercice : quelle est la probabilité de survie de Rose et Jack ?

Définir les booléens pour *femmes, classe1, classe2, classe3, survivant, ...*

In [ ]:
femmes = t.Sex=="female"
classe1 = t.Pclass == 1
classe2 = t.Pclass == 2
classe3 = t.Pclass == 3
survivant = t.Survived == 1
mort = ~ survivant

Jack est un homme en 3ème classe, et Rose une femme en 1ère (définir les booléens *jack* et *rose*) :

In [ ]:
jack = hommes & classe3
rose = femmes & classe1

Calculer la probabilité de survie de Jack :

In [ ]:
p_jack = t[jack & survivant].Sex.count()/t[jack].Sex.count()
print(p_jack)

Calculer la probabilité de survie de Rose :

In [ ]:
p_rose = t[rose & survivant].Sex.count()/t[rose].Sex.count()
print(p_rose)

## Exercice : tester différentes visualisations sur le dataset

Tracer différentes représentations du dataset

In [ ]:
t.head()

In [ ]:
t.columns

In [ ]:
t.describe()

In [ ]:
fig = sns.FacetGrid(t, hue="Survived", aspect=5, palette="Set2")
fig.map(sns.kdeplot, "Fare", shade=True)
fig.add_legend()

In [ ]:
sns.jointplot("Age", "Pclass", t, kind='kde');

Tracer les courbes de distribution de l'âge selon la classe (utiliser *FacetGrid*)

In [ ]:
sns.lmplot(x="Age", y="Pclass", data=t, fit_reg=False, hue='Survived')

In [ ]:
sns.boxplot("Pclass", "Age", data=t)

Que peut-on dire des voyageurs de 1ere, 2eme et 3eme classe ?

Les voyageurs de 1ere classe ont un age moyen autour de 40 ans, 2e classe de 30 ans et 3e classe de 25 ans.

## Conditionnement des données

Eliminer les colonnes non pertinentes pour la prédiction (on peut utiliser une liste de colonnes dans *drop*), et placer le résultat dans la variable *titanic* :

In [ ]:
t.columns

In [ ]:
# On élimine les colonnes non pertinentes pour la prédiction
titanic = t.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [ ]:
titanic.count()

Il manque des valeurs, par exemple pour la colonne *age*

Les valeurs inconnues sont affichées comme **NaN** (*Not a Number*).  
On peut tester si une valeur est **NaN** avec la fonction *np.isnan(valeur)*  
Afficher les lignes pour lesquelles l'âge est inconnu :

In [ ]:
titanic[np.isnan(titanic.Age)]

### Données manquantes

On voit qu'il manque des données, en particulier pour la colonne *'age'*  
Il existe plusieurs approches pour compléter les données manquantes :  
- **suppression** des données manquantes (par exemple avec la fonction *dropna*). C'est une méthode simple, mais qui élimine de l'information
- **remplacement** des données manquantes. Par exemple, on pourrait remplacer les informations manquantes pour l'âge par la moyenne de la colonne (mais on introduit un biais sur cette valeur), ou par un nombre aléatoire généré par une loi normale de même moyenne et variance ...
- **estimation** des paramètres manquants avec une méthode de prédiction (par exemple avec une régression)

La fonction *fillna* permet de compléter simplement les paramètres manquants. 

In [ ]:
titanic1 = titanic.fillna(value = {'Age':titanic.Age.mean()})

Tracer l'histogramme des âges. Qu'observez-vous ?

In [ ]:
plt.hist(titanic1.Age, bins=80)

In [ ]:
titanic = titanic.fillna(method='pad')

L'option *method='pad'* permet d'utiliser la précédente valeur non manquante :

In [ ]:
titanic = titanic.fillna(method='pad')

In [ ]:
titanic.count()

Tracer l'histogramme pour *age* :

In [ ]:
plt.hist(titanic.Age, bins=80)

La distribution des âges n'est pas significativement modifiée ...

## Déséquilibre des distributions

Certaines distributions sont déséquilibrées, et éloignées d'une loi normale :

In [ ]:
sns.distplot(titanic.Fare, color='blue')

Dans ce cas, une transformation log peut améliorer l'équilibre :

In [ ]:
titanic['log_fare'] = np.log(titanic.Fare+1)

In [ ]:
sns.kdeplot(titanic.log_fare, color='blue')

In [ ]:
titanic = titanic.drop(['Fare'], axis=1)

### Mise à l'échelle des données quantitatives

In [ ]:
titanic[['Age','log_fare']].describe()

In [ ]:
sns.kdeplot(titanic.log_fare, color='blue')
sns.kdeplot(titanic.Age, color='red')

On voit qu'il y a une forte différente de distribution entre les deux séries.  
Certains algorithmes demandent une distribution normalisée. Pour une discussion détaillée sur ce sujet, cf par exemple :  
http://www.faqs.org/faqs/ai-faq/neural-nets/part2/section-16.html  
http://scikit-learn.org/stable/modules/preprocessing.html

La librairie *sklearn* comporte une librairie de prétraitement des données

In [ ]:
from sklearn import preprocessing

On peut normaliser les valeurs min et à max (valeurs ramenées entre 0 et 1) :

In [ ]:
minmax = preprocessing.MinMaxScaler(feature_range=(0, 1))
titanic[['Age', 'log_fare']] = minmax.fit_transform(titanic[['Age', 'log_fare']])

In [ ]:
sns.distplot(titanic.log_fare, color='blue')
sns.distplot(titanic.Age, color='red')

On peut également utiliser le *StandardScaler* pour ramener la moyenne à 0 et l'écart type à 1 :

In [ ]:
scaler = preprocessing.StandardScaler()
titanic[['Age', 'log_fare']] = scaler.fit_transform(titanic[['Age', 'log_fare']])

In [ ]:
sns.kdeplot(titanic.log_fare, color='blue')
sns.kdeplot(titanic.Age, color='red')

### Encodage binaire des données qualitatives (*one hot encoding*)

In [ ]:
titanic.info()

La plupart des algorithmes ont besoin de données numériques, et n'acceptent pas les chaînes de caractères :

In [ ]:
titanic.Sex = titanic.Sex.map({"male":0, "female":1})

In [ ]:
titanic.head()

On utilise la fonction *get_dummies* de Pandas pour transformer les colonnes multimodales (par exemple 'embarked') en plusieurs colonnes binaires (par exemple 'embarked_C' dont les valeurs sont 1 si le passager a embarqué à Cherbourg et 0 sinon) :

In [ ]:
titanic = pd.get_dummies(data=titanic, columns=['Pclass', 'Embarked'])

In [ ]:
titanic.head()

## Création des jeux d'apprentissage et de test

Créer les jeux d'apprentissage et de test

In [ ]:
X = titanic.drop(['Survived'], axis=1)
y = titanic.Survived

On sépare le dataset en deux parties :
- un ensemble d'apprentissage (entre 70% et 90% des données), qui va permettre d'entraîner le modèle
- un ensemble de test (entre 10% et 30% des données), qui va permettre d'estimer la pertinence de la prédiction

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

## Régression logistique

Appliquer une régression logistique pour classifier sur l'ensemble de test

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)

## Mesures de performance

In [ ]:
# Importation des méthodes de mesure de performances
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

En comparant les valeurs prédites et les valeurs réelles, on a plusieurs possibilités :
- *Vrais positifs* (VP ou TP) : on prédit "oui" et la valeur attendue est "oui"
- *Vrais négatifs* (VN ou TN) : on prédit "non" et la valeur attendue est "non"
- *Faux positifs* (FP) : on prédit "oui" et la valeur attendue est "non"
- *Faux négatifs* (FN) : on prédit "non" et la valeur attendue est "oui"

Par exemple, si veut prédire le décès, le nombre de vrais positifs est le nombre de fois où on a prédit 0 pour des passagers effectivement morts sur le Titanic (*survived = 0*)

Afficher la matrice de confusion :

In [ ]:
print(confusion_matrix(y_test,y_lr))

La **matrice de confusion** permet de compter les vrais positifs, faux positifs, ...

<img src="https://i.stack.imgur.com/gKyb9.png">

La pertinence (ou *accuracy*) mesure le nombre de bonnes prédictions sur le nombre total d'observations

In [ ]:
print(accuracy_score(y_test,y_lr))

Néanmoins cette mesure peut être faussée dans certains cas, en particulier si le nombre de 0 et de 1 est déséquilibré.
On a donc d'autres estimateurs :
- la **précision** est le nombre de prédictions positives correctes sur le nombre total de prédictions positives : *precision = VP/(VP+FP)*
- la **sensibilité** (*recall*) est le nombre de prédictions positives sur le nombre effectif de "oui" : *recall = VP:(VP+FN)*
- le **score F1** est la moyenne pondérée de la précision et de la sensibilité : *f1-score = 2xprecisionxrecall/(precision+recall)*

In [ ]:
print(classification_report(y_test, y_lr))

*predict_proba* donne un tableau de couples de probabilités : *[probabilité de prédiction 0, probabilité de prédiction 1]*

In [ ]:
probas = lr.predict_proba(X_test)

In [ ]:
print(probas)

On met les probabilités de prédiction de la valeur 1 dans un dataframe, avec les valeurs effectives, pour faciliter la visualisation :

In [ ]:
dfprobas = pd.DataFrame(probas,columns=['proba_0','proba_1'])
dfprobas['y'] = np.array(y_test)

In [ ]:
dfprobas

On affiche la distribution des probabilités de prédiction de 1, et celle des non probabilités de prédiction de 0 :

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(1-dfprobas.proba_0[dfprobas.y==0], bins=50)
sns.distplot(dfprobas.proba_1[dfprobas.y==1], bins=50)

La distribution idéale permet de séparer totalement la prédiction des positifs et négatifs :  
<img src="https://miro.medium.com/max/660/1*Uu-t4pOotRQFoyrfqEvIEg.png">
Le cas le plus défavorable consiste en une distribution équivalente pour les positifs et les négatifs :  
<img src="https://miro.medium.com/max/538/1*iLW_BrJZRI0UZSflfMrmZQ.png">

On utilise ces distributions pour construire la **courbe ROC** (Receiving Operator Characteristic) qui représente le taux de vrais positifs par rapport aux taux de faux positifs.  
La mesure de l'aire sous la courbe **AUC** (Area Under Curve) est un bon indicateur de performance  
Pour plus de détails : http://www.xavierdupre.fr/app/mlstatpy/helpsphinx/c_metric/roc.html

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas[:, 1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

In [ ]:
plt.figure(figsize=(12,12))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')        # plus mauvaise courbe
plt.plot([0,0,1],[0,1,1],'g:')     # meilleure courbe
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

## Ajustement des hyperparamètres (Random Forests)

On teste les forêts aléatoires :

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

Parmi les hyperparamètres de l'algorithme qui peuvent avoir un impact sur les performances, on a :
- **n_estimators** : le nombre d'arbres de décision de la forêt aléatoire
- **min_samples_leaf** : le nombre d'échantillons minimum dans une feuille de chaque arbre
- **max_features** : le nombre de caractéristiques à prendre en compte lors de chaque split

Pour chaque algorithme de *sklearn*, on peut trouver la liste des paramètres dans la documentation, avec des exemples :  
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html  

In [ ]:
rf1 = ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=10, max_features=3)
rf1.fit(X_train, y_train)
y_rf1 = rf.predict(X_test)
print(classification_report(y_test, y_rf1))

*validation_curve* permet de tracer la courbe du score sur un ensemble d'apprentissage et sur un ensemble de test (*cross validation*), en faisant varier un paramètre, par exemple *n_estimators* :

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'n_estimators', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('n_estimators')
plt.ylabel('score');

**Exercice** : tracer les courbes de validation pour les paramètres *min_samples_leaf* et *max_features* (attention pour ce dernier, le nombre max est le nombre de caractéristiques / colonnes du tableau)

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'min_samples_leaf', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('min_samples_leaf')
plt.ylabel('score');

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, params/dfprobas.count(),step=30)
train_score, val_score = validation_curve(rf, X, y, 'max_features', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('max_features')
plt.ylabel('score');

La méthode *GridSearchCV* permet de tester plusieurs combinaisons de paramètres (listés dans une grille de paramètres) et de sélectionner celle qui donne la meilleure pertinence

In [ ]:
from sklearn import model_selection

In [ ]:
param_grid = {
              'n_estimators': [10, 100, 500],
              'min_samples_leaf': [1, 20, 50]
             }
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)

Ici on a choisi des valeurs pour le nombres d'arbres dans la forêt aléatoire (*'n_estimators'*) et le nombre minimum d'échantillons pour une feuille. On pourrait tester d'autres valeurs, et d'autres paramètres, cf :  
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

On lance l'entrainement :

In [ ]:
rf_gs.fit(X_train, y_train)

On peut voir les paramètres sélectionnés et le score :

In [ ]:
print(rf_gs.best_params_)

On sélectionne le meilleur estimateur :

In [ ]:
rf2 = rf_gs.best_estimator_

In [ ]:
y_rf2 = rf2.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf2))

On a amélioré la performance du modèle

### Importance des caractéristiques

L'attribut *feature_importances_* renvoie un tableau du poids de chaque caractéristique dans la décision :

In [ ]:
importances = rf2.feature_importances_
indices = np.argsort(importances)

On peut visualiser ces degrés d'importance avec un graphique à barres par exemple :

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caracteristiques')

## XGBoost

La méthode XGBoost est dérivée des arbres de décision, et très efficace, en particulier pour de grandes quantités de données.

Sous Anaconda prompt :
*pip install xgboost*  
(déjà disponible sous Kaggle)

In [ ]:
# Sous Jupyter, si xgboost n'est pas déjà installé
!pip install xgboost

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
cm = confusion_matrix(y_test, y_xgb)
print(cm)
print(classification_report(y_test, y_xgb))

## Exercice : explorer d'autres méthodes de classification

<img src = "http://scikit-learn.org/0.16/_static/ml_map.png">

http://scikit-learn.org/0.16/tutorial/machine_learning_map/index.html

## Exercice : appliquer les méthodes sur le dataset *Indian Diabete*

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# Lecture des données d'apprentissage et de test
d = pd.read_csv("../input/pyms-diabete/diabete.csv")

In [ ]:
d.head()

In [ ]:
d.count()

In [ ]:
d.glucose.value_counts()

In [ ]:
d.n_pregnant.value_counts()

In [ ]:
d.diabete.value_counts()

In [ ]:
#jamaisEnceinte = d.n_pregnant == 0
dejaEnceinte = d.n_pregnant >= 1
#glucosePlusDe100 = d.glucose >= 100

#test_jamaisEnceinte_et_glucocePlusDe100 = jamaisEnceinte & glucosePlusDe100

In [ ]:
# p_test_jamaisEnceinte_et_diabetique = d[jamaisEnceinte & d.diabete].age.count()/d.age.count()
# print('Si t\'as jamais été enceinte et diabetique :')
# print(p_test_jamaisEnceinte_et_diabetique)

p_test_dejaEnceinte_et_diabetique = d[dejaEnceinte & d.diabete].age.count()/d.age.count()
print('Si t\'as déjà été enceinte et diabetique :')
print(p_test_dejaEnceinte_et_diabetique)

# p_test_glucocePlusDe100_et_diabetique = d[glucosePlusDe100 & d.diabete].age.count()/d.age.count()
# print('Si tu as un glucose de plus de 100 et diabetique :')
# print(p_test_glucocePlusDe100_et_diabetique)

# p_test_jamaisEnceinte_et_glucocePlusDe100 = d[test_jamaisEnceinte_et_glucocePlusDe100 & d.diabete].age.count()/d.age.count()
# print('Si t\'as jamais été enceinte et tu as un glucose de plus de 100 et diabetique :')
# print(p_test_jamaisEnceinte_et_glucocePlusDe100)


In [ ]:
sns.lmplot(x="insulin", y="pedigree", data=d, fit_reg=False, hue='diabete')

In [ ]:
sns.jointplot("insulin", "glucose", d, kind='kde');

In [ ]:
d.columns

In [ ]:
# On élimine les colonnes non pertinentes pour la prédiction
# diabet = d.drop(['age'], axis=1)

In [ ]:
sns.distplot(d.n_pregnant, color='blue')

In [ ]:
sns.distplot(d.glucose, color='blue')

In [ ]:
sns.distplot(d.tension, color='blue')

In [ ]:
sns.distplot(d.thickness, color='blue')

In [ ]:
sns.distplot(d.insulin, color='blue')

In [ ]:
sns.distplot(d.bmi, color='blue')

In [ ]:
sns.distplot(d.pedigree, color='blue')

In [ ]:
d['log_thickness'] = np.log(d.thickness+1)

In [ ]:
sns.distplot(d.log_thickness, color='blue')

In [ ]:
d['log_pedigree'] = np.log(d.pedigree+1)

In [ ]:
sns.distplot(d.log_pedigree, color='blue')

In [ ]:
d = d.drop(['pedigree', 'thickness'], axis=1)

In [ ]:
d.count()

In [ ]:
sns.kdeplot(d.n_pregnant, color="blue")
sns.kdeplot(d.bmi, color="red")

In [ ]:
d.info()

In [ ]:
X = d.drop(['diabete'], axis=1)
y = d.diabete

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)

In [ ]:
# Importation des méthodes de mesure de performances
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score


In [ ]:
print(confusion_matrix(y_test,y_lr))

In [ ]:
print(accuracy_score(y_test,y_lr))

In [ ]:
#RandomForest

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

In [ ]:
rf1 = ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=10, max_features=3)
rf1.fit(X_train, y_train)
y_rf1 = rf.predict(X_test)
print(classification_report(y_test, y_rf1))

In [ ]:
from sklearn import model_selection

In [ ]:
param_grid = {
              'n_estimators': [10, 100, 500],
              'min_samples_leaf': [1, 20, 50]
             }
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)

In [ ]:
rf_gs.fit(X_train, y_train)

In [ ]:
print(rf_gs.best_params_)

In [ ]:
rf2 = rf_gs.best_estimator_

In [ ]:
y_rf2 = rf2.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf2))

In [ ]:
importances = rf2.feature_importances_
indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caracteristiques')